In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf

# --- 1. ADATOK BETÖLTÉSE ---
print("Adatok betöltése...")
try:
    # Feltételezzük, hogy a stock_market_final_panel.csv-ben a PAST már a 20 napos átlag
    df = pd.read_csv("stock_market_final_panel.csv")
    df['Date'] = pd.to_datetime(df['Date'])
    print(f"Sikeres betöltés: {len(df)} sor.")
except FileNotFoundError:
    print("HIBA: Nem találom a 'stock_market_final_panel.csv' fájlt!")
    exit()

# --- 2. BENCHMARK (ACWI) ELŐKÉSZÍTÉSE ---
print("Benchmark (ACWI) letöltése...")
try:
    acwi = yf.download("ACWI", start="2021-06-01", end="2022-04-01", progress=False)
    if not acwi.empty:
        # Árfolyam kinyerése (Close)
        price = acwi['Close'].iloc[:, 0] if (isinstance(acwi.columns, pd.MultiIndex) and 'Close' in acwi.columns) else acwi['Close'] if 'Close' in acwi.columns else acwi.iloc[:, 0]
        if isinstance(price, pd.DataFrame): price = price.squeeze()
        
        # Loghozam számítása
        market_return = np.log(price / price.shift(1)).rename("Market_Return")
        print("✅ Benchmark OK.")
    else:
        raise ValueError("Üres adat")
except:
    print("⚠️ Benchmark letöltés sikertelen, országok átlagát használjuk proxyként.")
    market_return = df.groupby('Date')['Return'].mean().rename("Market_Return")

# --- 3. CAR SZÁMÍTÁSA (Event Study) ---
EVENT_DATE = pd.Timestamp("2022-02-24")
results = []
countries = df['Country'].unique()

print(f"CAR számítása {len(countries)} országra...")

for country in countries:
    try:
        # Adatok szűrése az adott országra
        c_data = df[df['Country'] == country].set_index('Date').sort_index()
        
        # Összefésülés a Benchmarkkal
        merged = pd.concat([c_data['Return'], market_return], axis=1).dropna()
        
        if EVENT_DATE not in merged.index: continue
        loc = merged.index.get_loc(EVENT_DATE)
        
        # Becslési ablak: -140 naptól -6 napig
        est = merged.iloc[loc-140 : loc-6]
        if len(est) < 30: continue
        
        # Market Model becslése (Alpha, Beta)
        model = sm.OLS(est['Return'], sm.add_constant(est['Market_Return'])).fit()
        alpha, beta = model.params['const'], model.params['Market_Return']
        
        # CAR számítása [+1, +3] ablakra (vagy amit a cikk kér)
        # Az ablak indexei: loc+1 -től loc+3-ig (a szeletelés vége nem inkluzív, ezért +4)
        win_data = merged.iloc[loc+1 : loc+4]
        
        expected_ret = alpha + beta * win_data['Market_Return']
        abnormal_ret = win_data['Return'] - expected_ret
        car = abnormal_ret.sum() * 100 # Százalékban
        
        # Metaadatok a regresszióhoz (az első sorból vesszük ki a fix értékeket)
        meta = c_data.iloc[0]
        
        results.append({
            'Country': country,
            'CAR': car,
            'NATO': meta['NATO'],
            'DEV': meta['DEV'],
            'TGDP': meta['TRADE'],   # A képen ez TGDP
            'PAST': meta['PAST'] * 100 if pd.notnull(meta['PAST']) else 0, # Sárgával jelölt változó
            'EXRATE': meta['EXRATE']
        })
    except Exception as e:
        pass

# --- 4. KERESZTMETSZETI REGRESSZIÓ (Cross-Sectional Regression) ---
# Ez felel meg a képen látható egyenletnek
reg_df = pd.DataFrame(results).dropna()

print("\n" + "="*60)
print("TABLE 5 REPRODUKCIÓ (A feltöltött képlet alapján)")
print("Equation: CAR = α + β1*NATO + β2*DEV + β3*TGDP + β4*PAST + β5*EXRATE")
print("="*60)

if len(reg_df) > 10:
    # Független változók (X)
    X_vars = ['NATO', 'DEV', 'TGDP', 'PAST', 'EXRATE']
    X = reg_df[X_vars]
    X = sm.add_constant(X) # Konstans (α) hozzáadása
    
    # Függő változó (Y)
    Y = reg_df['CAR']
    
    # Modell futtatása (Robust Standard Errors - HC1)
    model_reg = sm.OLS(Y, X).fit(cov_type='HC1')
    
    print(model_reg.summary())
    
    # Értékelés
    print("\n--- Eredményelemzés ---")
    nato_p = model_reg.pvalues['NATO']
    if nato_p < 0.05:
        print(f"✅ A NATO hatás szignifikáns! (p={nato_p:.4f})")
    else:
        print(f"❌ A NATO hatás nem szignifikáns. (p={nato_p:.4f})")
else:
    print("HIBA: Túl kevés adat áll rendelkezésre a regresszióhoz.")

Adatok betöltése...
Sikeres betöltés: 33690 sor.
Benchmark (ACWI) letöltése...


C:\Users\andre\AppData\Local\Temp\ipykernel_3716\1809934984.py:20: FutureWarning: YF.download() has changed argument auto_adjust default to True
  acwi = yf.download("ACWI", start="2021-06-01", end="2022-04-01", progress=False)


✅ Benchmark OK.
CAR számítása 46 országra...

TABLE 5 REPRODUKCIÓ (A feltöltött képlet alapján)
Equation: CAR = α + β1*NATO + β2*DEV + β3*TGDP + β4*PAST + β5*EXRATE
                            OLS Regression Results                            
Dep. Variable:                    CAR   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     2.854
Date:                Wed, 17 Dec 2025   Prob (F-statistic):             0.0277
Time:                        20:42:05   Log-Likelihood:                -105.44
No. Observations:                  44   AIC:                             222.9
Df Residuals:                      38   BIC:                             233.6
Df Model:                           5                                         
Covariance Type:                  HC1                                         
                 coef    std err          z  

In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf

# --- 1. BEÁLLÍTÁSOK ÉS ADATBETÖLTÉS ---
EVENT_DATE = pd.Timestamp("2022-02-24")
WINDOWS = [(-5, -1), (-3, -1), (0, 0), (1, 3), (1, 5)]
WINDOW_NAMES = ['[-5,-1]', '[-3,-1]', '[0, 0]', '[+1,+3]', '[+1,+5]']

print("Adatok betöltése...")
try:
    df = pd.read_csv("stock_market_final_panel.csv")
    df['Date'] = pd.to_datetime(df['Date'])
except:
    print("HIBA: Nem találom a 'stock_market_final_panel.csv' fájlt.")
    exit()

# Benchmark (ACWI) letöltése vagy Proxy
print("Benchmark előkészítése...")
try:
    acwi = yf.download("ACWI", start="2021-06-01", end="2022-04-01", progress=False)
    if not acwi.empty:
        price = acwi['Close'].iloc[:, 0] if (isinstance(acwi.columns, pd.MultiIndex) and 'Close' in acwi.columns) else acwi['Close'] if 'Close' in acwi.columns else acwi.iloc[:, 0]
        if isinstance(price, pd.DataFrame): price = price.squeeze()
        market_return = np.log(price / price.shift(1)).rename("Market_Return")
    else: raise ValueError
except:
    market_return = df.groupby('Date')['Return'].mean().rename("Market_Return")

# --- 2. CAR SZÁMÍTÁS MINDEN ABLAKRA ---
results = []
countries = df['Country'].unique()

print(f"Elemzés futtatása {len(countries)} országra...")

for country in countries:
    try:
        c_data = df[df['Country'] == country].set_index('Date').sort_index()
        merged = pd.concat([c_data['Return'], market_return], axis=1).dropna()
        
        if EVENT_DATE not in merged.index: continue
        loc = merged.index.get_loc(EVENT_DATE)
        
        # Market Model (-140-től -6-ig)
        est = merged.iloc[loc-140 : loc-6]
        if len(est) < 30: continue
        
        model = sm.OLS(est['Return'], sm.add_constant(est['Market_Return'])).fit()
        alpha, beta = model.params['const'], model.params['Market_Return']
        
        # Adatok összegyűjtése az országhoz
        row = {
            'Country': country,
            'NATO': c_data.iloc[0]['NATO'],
            'DEV': c_data.iloc[0]['DEV'],
            'TGDP': c_data.iloc[0]['TRADE'],
            'EXRATE': c_data.iloc[0]['EXRATE'],
            'PAST': c_data.iloc[0]['PAST'] * 100 if pd.notnull(c_data.iloc[0]['PAST']) else 0
        }
        
        # CAR számítása minden ablakra
        for start, end in WINDOWS:
            # Indexek: loc + start -tól loc + end + 1 -ig
            w_data = merged.iloc[loc + start : loc + end + 1]
            expected = alpha + beta * w_data['Market_Return']
            abnormal = w_data['Return'] - expected
            # CAR százalékban
            col_name = f"CAR_{start}_{end}"
            row[col_name] = abnormal.sum() * 100
            
        results.append(row)
    except: pass

data = pd.DataFrame(results).dropna()

# --- 3. REGRESSZIÓK ÉS TÁBLÁZAT ÉPÍTÉS (TABLE 5) ---
final_table_data = {}
indep_vars = ['NATO', 'DEV', 'TGDP', 'PAST', 'EXRATE']
X = sm.add_constant(data[indep_vars])

print("\nRegressziók futtatása...")

for i, (start, end) in enumerate(WINDOWS):
    dep_var = f"CAR_{start}_{end}"
    Y = data[dep_var]
    
    # OLS Futtatás
    model = sm.OLS(Y, X).fit(cov_type='HC1')
    
    # Eredmények formázása ebbe az oszlopba
    col_results = []
    
    # Változók sorai
    for var in indep_vars:
        coef = model.params[var]
        p_val = model.pvalues[var]
        t_stat = model.tvalues[var]
        
        stars = "***" if p_val < 0.01 else "**" if p_val < 0.05 else "*" if p_val < 0.1 else ""
        
        # Két sor per változó: Koefficiens és (t-stat)
        col_results.append(f"{coef:.4f}{stars}")
        col_results.append(f"({t_stat:.2f})")
        
    # Statisztikák az aljára
    col_results.append(f"{model.rsquared_adj:.3f}") # Adj. R2
    col_results.append(f"{model.fvalue:.2f}{'***' if model.f_pvalue < 0.01 else ''}") # F-stat
    col_results.append(int(model.nobs)) # Obs
    
    final_table_data[WINDOW_NAMES[i]] = col_results

# --- 4. TÁBLÁZAT MEGJELENÍTÉSE ---
# Sorok nevei
row_names = []
for var in indep_vars:
    row_names.append(var)
    row_names.append("") # Üres sor a t-statnak
row_names.extend(["Adjusted R2", "F-statistic", "Observations"])

# DataFrame létrehozása
table_5 = pd.DataFrame(final_table_data, index=row_names)

print("\n" + "="*65)
print("TABLE 5 REPRODUKCIÓ: Cross-sectional analysis of CARs")
print("="*65)
print(table_5)
print("="*65)
print("Megjegyzés: A zárójelben lévő értékek a t-statisztikák.")
print("*** p<0.01, ** p<0.05, * p<0.1")

Adatok betöltése...
Benchmark előkészítése...


C:\Users\andre\AppData\Local\Temp\ipykernel_3716\3229410156.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  acwi = yf.download("ACWI", start="2021-06-01", end="2022-04-01", progress=False)


Elemzés futtatása 46 országra...

Regressziók futtatása...

TABLE 5 REPRODUKCIÓ: Cross-sectional analysis of CARs
              [-5,-1]     [-3,-1]   [0, 0]     [+1,+3]   [+1,+5]
NATO          -0.5167      0.3514  -1.1717     -1.3965   -1.0089
              (-0.94)      (0.96)  (-1.35)     (-0.99)   (-0.65)
DEV            0.2237      0.0761   0.8155      0.4210  -1.9240*
               (0.42)      (0.19)   (0.93)      (0.32)   (-1.81)
TGDP          -0.0046  -0.0053***  -0.0060     -0.0066   -0.0077
              (-1.41)     (-3.09)  (-1.62)     (-1.26)   (-1.60)
PAST          3.0870*    2.4389**  4.1467*    7.5157**  6.4317**
               (1.86)      (2.26)   (1.70)      (1.98)    (2.04)
EXRATE         0.0000      0.0001   0.0000  -0.0002***   -0.0002
               (0.67)      (1.39)   (0.27)     (-2.80)   (-1.54)
Adjusted R2     0.079       0.238    0.113       0.156     0.247
F-statistic      2.94     6.08***     2.20        2.85   9.29***
Observations       44          44       4

In [13]:
# --- MINTA ELLENŐRZÉSE ---

# 1. A regresszióba bekerült országok lekérése
# (A kódodban 'reg_df' vagy 'res_df' a neve a tisztított táblának)
if 'reg_df' in locals():
    final_data = reg_df
elif 'res_df' in locals():
    final_data = res_df
else:
    print("Hiba: Nem találom a regressziós táblát. Futtasd le előbb a regressziót!")
    final_data = pd.DataFrame()

if not final_data.empty:
    included_list = final_data['Country'].unique()
    included_list.sort() # Ábécé sorrendbe rendezzük
    
    print("="*50)
    print(f"A MODELLBE BEKERÜLT ORSZÁGOK LISTÁJA (N={len(included_list)})")
    print("="*50)
    
    for i, country in enumerate(included_list, 1):
        print(f"{i}. {country}")
        
    # 2. Megnézzük, kik maradtak ki
    all_countries = df['Country'].unique()
    excluded_list = list(set(all_countries) - set(included_list))
    
    if excluded_list:
        print("\n" + "-"*50)
        print(f"KIMARADT ({len(excluded_list)} db):")
        for c in excluded_list:
            print(f"❌ {c}")
    else:
        print("\n✅ Minden ország bekerült!")

A MODELLBE BEKERÜLT ORSZÁGOK LISTÁJA (N=44)
1. Australia
2. Austria
3. Belgium
4. Brazil
5. Canada
6. Chile
7. China
8. Colombia
9. Czech Republic
10. Egypt
11. Finland
12. France
13. Germany
14. Greece
15. Hong Kong
16. Hungary
17. India
18. Indonesia
19. Ireland
20. Israel
21. Italy
22. Japan
23. Kuwait
24. Malaysia
25. Mexico
26. Netherlands
27. New Zealand
28. Norway
29. Peru
30. Philippines
31. Portugal
32. Qatar
33. Saudi Arabia
34. Singapore
35. South Africa
36. South Korea
37. Spain
38. Sweden
39. Switzerland
40. Thailand
41. Turkey
42. UAE
43. UK
44. US

--------------------------------------------------
KIMARADT (2 db):
❌ Taiwan
❌ Denmark
